In [55]:
import numpy as np
import pandas as pd

import json
import csv

import datetime

In [109]:
# this is where we will only use what we need.

filename = 'ECE219_tweet_data/tweets_#gohawks.txt'
output_filename = 'Q1_2#gohawks.lol'

with open(filename, 'r') as reader:
    headers = ['citation_date','tweets','retweets','followers']
    with open(output_filename, 'w') as writer:
        csv_writer = csv.writer(writer, lineterminator='\n')
        csv_writer.writerow(headers)
        for line in reader:
            data = json.loads(line)
            response = [datetime.datetime.fromtimestamp(data['citation_date']),
                        1, # there is one tweet at a time..
                        data['metrics']['citations']['total'], #retweets
                        data['author']['followers'], #followers
                         ]
            csv_writer.writerow(response)
            
            
            
filename = 'ECE219_tweet_data/tweets_#gopatriots.txt'
output_filename = 'Q1_2#gopatriots.lol'

with open(filename, 'r') as reader:
    headers = ['citation_date','tweets','retweets','followers']
    with open(output_filename, 'w') as writer:
        csv_writer = csv.writer(writer, lineterminator='\n')
        csv_writer.writerow(headers)
        for line in reader:
            data = json.loads(line)
            response = [datetime.datetime.fromtimestamp(data['citation_date']),
                        1, # there is one tweet at a time..
                        data['metrics']['citations']['total'], #retweets
                        data['author']['followers'], #followers
                         ]
            csv_writer.writerow(response)

            
filename = 'ECE219_tweet_data/tweets_#nfl.txt'
output_filename = 'Q1_2#nfl.lol'

with open(filename, 'r') as reader:
    headers = ['citation_date','tweets','retweets','followers']
    with open(output_filename, 'w') as writer:
        csv_writer = csv.writer(writer, lineterminator='\n')
        csv_writer.writerow(headers)
        for line in reader:
            data = json.loads(line)
            response = [datetime.datetime.fromtimestamp(data['citation_date']),
                        1, # there is one tweet at a time..
                        data['metrics']['citations']['total'], #retweets
                        data['author']['followers'], #followers
                         ]
            csv_writer.writerow(response)

            
filename = 'ECE219_tweet_data/tweets_#patriots.txt'
output_filename = 'Q1_2#patriots.lol'

with open(filename, 'r') as reader:
    headers = ['citation_date','tweets','retweets','followers']
    with open(output_filename, 'w') as writer:
        csv_writer = csv.writer(writer, lineterminator='\n')
        csv_writer.writerow(headers)
        for line in reader:
            data = json.loads(line)
            response = [datetime.datetime.fromtimestamp(data['citation_date']),
                        1, # there is one tweet at a time..
                        data['metrics']['citations']['total'], #retweets
                        data['author']['followers'], #followers
                         ]
            csv_writer.writerow(response)
      
    
filename = 'ECE219_tweet_data/tweets_#sb49.txt'
output_filename = 'Q1_2#sb49.lol'

with open(filename, 'r') as reader:
    headers = ['citation_date','tweets','retweets','followers']
    with open(output_filename, 'w') as writer:
        csv_writer = csv.writer(writer, lineterminator='\n')
        csv_writer.writerow(headers)
        for line in reader:
            data = json.loads(line)
            response = [datetime.datetime.fromtimestamp(data['citation_date']),
                        1, # there is one tweet at a time..
                        data['metrics']['citations']['total'], #retweets
                        data['author']['followers'], #followers
                         ]
            csv_writer.writerow(response)
            
filename = 'ECE219_tweet_data/tweets_#superbowl.txt'
output_filename = 'Q1_2#superbowl.lol'

with open(filename, 'r') as reader:
    headers = ['citation_date','tweets','retweets','followers']
    with open(output_filename, 'w') as writer:
        csv_writer = csv.writer(writer, lineterminator='\n')
        csv_writer.writerow(headers)
        for line in reader:
            data = json.loads(line)
            response = [datetime.datetime.fromtimestamp(data['citation_date']),
                        1, # there is one tweet at a time..
                        data['metrics']['citations']['total'], #retweets
                        data['author']['followers'], #followers
                         ]
            csv_writer.writerow(response)

In [74]:
# QUESTION 1: Report the following statistics for each hashtag:
#  Average number of tweets per hour
#  Average number of followers of users posting the tweets per tweet (to make it simple, we
# average over the number of tweets; if a users posted twice, we count the user and the user's
# followers twice as well)
#  Average number of retweets per tweet

In [111]:
gohawks = pd.read_csv('Q1_2#gohawks.lol',delimiter=',', parse_dates=['citation_date'])
gopatriots = pd.read_csv('Q1_2#gopatriots.lol',delimiter=',', parse_dates=['citation_date'])
nfl = pd.read_csv('Q1_2#nfl.lol',delimiter=',', parse_dates=['citation_date'])
patriots = pd.read_csv('Q1_2#patriots.lol',delimiter=',', parse_dates=['citation_date'])
sb49 = pd.read_csv('Q1_2#sb49.lol',delimiter=',', parse_dates=['citation_date'])
superbowl = pd.read_csv('Q1_2#superbowl.lol',delimiter=',', parse_dates=['citation_date'])

In [112]:
full_df = pd.concat([gohawks, gopatriots, nfl, patriots, sb49, superbowl])

In [113]:
max = full_df.max()
min = full_df.min()

In [104]:
test = (max['citation_date'] - min['citation_date']).total_seconds()

In [108]:
test.total_seconds()

2081588.0000000002

In [ ]:
# QUESTION 2: Plot \number of tweets in hour" over time for #SuperBowl and #NFL (a histogram
# with 1-hour bins). The tweets are stored in separate les for different hashtags and les are named
# as tweet [#hashtag].txt.